Installing the BeautifulSoup4 and lxml packages

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4
!conda install --yes --prefix {sys.prefix} lxml

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Importing the necessary packages/libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Getting the table from the Wikipedia page

In [3]:
source= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source, 'lxml')
data_table=soup.find('table', class_="wikitable sortable")

Converting the html table into a Pandas dataframe

In [4]:
table_rows = data_table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    #row = [tr.text for tr in td]
    #l.append(row)
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)
raw_table=pd.DataFrame(l, columns=["PostCode", "Borough", "Neighbourhood"])
raw_table.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping the rows with borough not assigned

In [5]:
#dropping the rows with borough not assigned
raw_table.drop(raw_table[raw_table.Borough=="Not assigned"].index, inplace=True)
raw_table.reset_index(drop=True, inplace=True)
raw_table.shape

(211, 3)

For 'Not assigned' neighbourhoods, making their Borough as their Neighbourhood

In [6]:
#for Not Assigned neighbourhoods, making their Borough as their Neighbourhood
for index, row in raw_table.iterrows():
    if row['Neighbourhood']=="Not assigned":
        row['Neighbourhood']=row['Borough']
raw_table.head(20)

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Merging rows with the same Post Codes

In [7]:
#raw_table,merging rows
final_table=raw_table.groupby(['PostCode','Borough'],as_index=False, sort=False).agg(','.join)
final_table['Neighbourhood']=set(final_table['Neighbourhood'])
final_table.head()


,PostCode,Borough,Neighbourhood
0,M3A,North York,"Brockton,Exhibition Place,Parkdale Village"
1,M4A,North York,"Ryerson,Garden District"
2,M5A,Downtown Toronto,"High Park,The Junction South"
3,M6A,North York,Central Bay Street
4,M7A,Queen's Park,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"


Displaying the final  shape of the dataframe

In [8]:
final_table.shape

(103, 3)

Importing the coordinates

In [9]:
#final_table.to_csv('PostCode_Processed_Table.csv',index = None, header=True)
coordinates=pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the 2 dataframes

In [10]:
final_table.rename(columns={'PostCode': 'Postal Code'}, inplace=True)
final_table.head()
loc_dataset=pd.merge(final_table, coordinates, on='Postal Code')
loc_dataset.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Brockton,Exhibition Place,Parkdale Village",43.753259,-79.329656
1,M4A,North York,"Ryerson,Garden District",43.725882,-79.315572
2,M5A,Downtown Toronto,"High Park,The Junction South",43.654260,-79.360636
3,M6A,North York,Central Bay Street,43.718518,-79.464763
4,M7A,Queen's Park,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",43.662301,-79.389494
5,M9A,Etobicoke,Leaside,43.667856,-79.532242
6,M1B,Scarborough,"Harbourfront East,Toronto Islands,Union Station",43.806686,-79.194353
7,M3B,North York,"Harbourfront,Regent Park",43.745906,-79.352188
8,M4B,East York,Studio District,43.706397,-79.309937
9,M5B,Downtown Toronto,"Clairlea,Golden Mile,Oakridge",43.657162,-79.378937
